# CatBoost HyperParameter Tuning

## Libraries

In [40]:
import os
os.chdir("../input")

import numpy as np
import pandas as pd
import optuna

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

## Helper Functions

In [ ]:
def objective(n_trials):
    params = {
        
    }

## Reading & Preparing Data

In [33]:
df = pd.read_csv("heart.csv")

In [38]:
X = df.drop("output", axis=1)
y = df["output"]

## Hyperparameter Tuning